The dataset contains an ID column, 25 features and one target label column. The description of the features is given below:

    url : URL of the webpage to be classified
    webpageDescription : One line description of the webpage
    alchemy_category : Alchemy category (per the publicly available Alchemy API)
    alchemy_category_score : Alchemy category score (per the publicly available Alchemy API)
    avgLinkWordLength : Average number of words in a webpage
    AvglinkWithOneCommonWord : Average number of web pages sharing at least one word with one other web page
    AvglinkWithTwoCommonWord : Average number of web pages sharing at least one word with two other web pages
    AvglinkWithThreeCommonWord : Average number of web pages sharing at least one word with three other web pages
    AvglinkWithFourCommonWord : Average number of web pages sharing at least one word with four other web pages
    redundancyMeasure : Measure of redundancy computed by finding the compression achieved on this web page via gzip
    embedRatio : Count of tags
    frameBased : Binary indication of whether a webpage has frameset markup
    frameTagRatio : Ratio of frameset markups over total markups
    domainLink: Binary indication of whether the webpage contains in URL with domain
    tagRatio : Ratio of tags over text in the webpage
    imageTagRatio : Ratio of tags over text in the webpage
    isNews : Binary indication of whether a webpage is a news article
    lengthyDomain : Binary indication of whether webpage's text contains more than 30 alpha-numeric characters
    hyperlinkToAllWordsRatio : Percentage of words on the webpage that are also in the hyperlink text
    isFrontPageNews : Binary indication of whether webpage is front-page news
    alphanumCharCount : Number of alpha-numeric characters in webpage's text
    linksCount : Number of markups
    wordCount : Number of words in URL
    parametrizedLinkRatio :
    spellingErrorsRatio : Ration of words that contain spelling errors

The label value of 0 represents that the webpage is not "ad-worthy", and a label value of 1 represents that the webpage is "ad-worthy". Your task is to predict the label value based on the features described above.

In [16]:
import numpy as np
import pandas as pd
import json

In [17]:
train_data = pd.read_csv('dataset/train_data.csv')
test_data = pd.read_csv('dataset/test_data.csv')

### Duplicate rows

In [18]:
print(train_data['url'].duplicated().sum())
print(test_data.duplicated().sum())

0
0


No duplicate rows

### Merging train and test data

### Converting webpageDescription from string to JSON data

In [19]:
merged_data = pd.concat([train_data, test_data])

merged_data['webpageDescription'] = merged_data['webpageDescription'].apply(lambda x: json.loads(x))

### Checking missing entries in each column

In [20]:
missing_entries = []

for feature in merged_data.columns:
    record = {}
    record['feature'] = feature
    record['NA values'] = merged_data[feature].isna().sum()
    record['? values'] = len(merged_data[merged_data[feature] == '?'][feature])
    missing_entries.append(record)

pd.DataFrame(missing_entries)

,feature,NA values,? values
0,url,0,0
1,webpageDescription,0,0
2,alchemy_category,0,2342
3,alchemy_category_score,0,2342
4,avgLinkWordLength,0,0
5,AvglinkWithOneCommonWord,0,0
6,AvglinkWithTwoCommonWord,0,0
7,AvglinkWithThreeCommonWord,0,0
8,AvglinkWithFourCommonWord,0,0
9,redundancyMeasure,0,0


### AlchemyCategory & AlchemyCategoryScore has 2342 missing entries (? values)

### isNews 2843 entries (? values)

### isFrontpageNews 1248 entries (? values)



### Analyzing the keys in JSON value of webpageDescription

It seems that "body" key is the one-line description of the page as described in the dataset description

In [21]:
merged_data['webpageDescription'].apply(lambda x: x.keys())

0       (title, body, url)
1            (body, title)
2       (title, body, url)
3       (title, body, url)
4       (url, title, body)
               ...        
1474    (title, body, url)
1475    (title, body, url)
1476    (url, title, body)
1477    (title, body, url)
1478    (title, body, url)
Name: webpageDescription, Length: 7395, dtype: object

In [22]:
len(merged_data[merged_data['webpageDescription'].apply(lambda x: x['body'] == None)])

57

### There are 57 entries where the webpageDescription has no "body" entry

In [23]:
merged_data[merged_data['webpageDescription'].apply(lambda x: x['title'] if x['body'] == None else x['body']).isna()]

,url,webpageDescription,alchemy_category,alchemy_category_score,avgLinkWordLength,AvglinkWithOneCommonWord,AvglinkWithTwoCommonWord,AvglinkWithThreeCommonWord,AvglinkWithFourCommonWord,redundancyMeasure,...,lengthyDomain,hyperlinkToAllWordsRatio,isFrontPageNews,alphanumCharCount,linksCount,wordCount,parametrizedLinkRatio,spellingErrorsRatio,label,id
2994,http://icanhascheezburger.com/2009/05/07/funny...,"{'title': None, 'body': None, 'url': 'icanhasc...",?,?,1.982178,0.756809,0.285992,0.235409,0.210117,0.0,...,1,35,?,8682,514,6,0.206226,0.125,0.0,5330


### There is 1 entry where the webpageDescription has no "body" entry and no "title" entry too

This entry belongs to train set, so we can just delete it

### Replacing webpageDescription using the following logic

    All entries have "body" key in webpageDescription column but some have these values as None
    So we fill in those entries with "title" key value
    
    if webpageDescription["body"] != None
        webpageDescription["body"]
    else webpageDescription["title"]

In [24]:
merged_data[merged_data['webpageDescription'].apply(lambda x: x['title'] if x['body'] == None else x['body']).isna()]

cleaned_data = merged_data.copy(deep=True)
cleaned_data.drop(index=2994, inplace=True)

In [25]:
cleaned_data['webpageDescription'] = cleaned_data['webpageDescription'].apply(lambda x: x['title'] if x['body'] == None else x['body'])

### Analyzing value_counts() of the two news columns

In [26]:
cleaned_data['isNews'].value_counts()

1    4552
?    2842
Name: isNews, dtype: int64

It seems pretty reasonable to assume that ? refers to isNews = 0 but more analysis needs to be done

In [27]:
cleaned_data['isFrontPageNews'].value_counts()

0    5853
?    1247
1     294
Name: isFrontPageNews, dtype: int64

It also seems reasonable to assume that ? refers to isFrontpageNews = 1 but more analysis needs to be done

In [28]:
cleaned_data[cleaned_data['isNews'] == '?'].head(10)

,url,webpageDescription,alchemy_category,alchemy_category_score,avgLinkWordLength,AvglinkWithOneCommonWord,AvglinkWithTwoCommonWord,AvglinkWithThreeCommonWord,AvglinkWithFourCommonWord,redundancyMeasure,...,lengthyDomain,hyperlinkToAllWordsRatio,isFrontPageNews,alphanumCharCount,linksCount,wordCount,parametrizedLinkRatio,spellingErrorsRatio,label,id
0,http://www.polyvore.com/cgi/home?id=1389651,polyvore is the best place to discover or sta...,?,?,1.916667,0.047619,0.007937,0.000000,0.000000,0.803797,...,0,34,0,682,126,1,0.531746,0.142857,1.0,3711
1,http://www.youtube.com/watch?v=ippMPPu6gh4,Speed Air Man--David Belle david belle speed a...,?,?,1.257576,0.141026,0.000000,0.000000,0.000000,1.142857,...,0,12,0,3008,78,1,0.628205,0.000000,1.0,7222
4,http://recipes.wuzzle.org/index.php/72,Barbecued Chicken Chow Siew from The Exotic Ki...,computer_internet,0.535009,0.181818,0.036364,0.000000,0.000000,0.000000,0.292614,...,0,3,0,1745,55,1,0.072727,0.115044,1.0,4321
6,http://www.insidershealth.com/natural_cure/poi...,Copyright 2011 InsidersHealth com All Rights R...,health,0.893828,1.691781,0.469799,0.147651,0.026846,0.006711,0.522134,...,0,57,0,961,149,4,0.006711,0.086957,1.0,5416
7,http://yawoot.com/post/2122,Well the world is made of peanut butter now po...,recreation,0.550131,1.170213,0.634921,0.000000,0.000000,0.000000,0.224906,...,0,11,0,1865,63,1,0.031746,0.123209,0.0,7349
16,http://www.nationalpost.com/news/story.html?id...,In a discovery that has stunned even those beh...,health,0.603181,3.261649,0.653595,0.369281,0.196078,0.140523,0.442116,...,1,44,0,5791,306,2,0.075163,0.098795,0.0,4794
17,http://thecompletecookbook.wordpress.com/,Before we get to today s post please understan...,science_technology,0.564931,1.371717,0.475904,0.269076,0.128514,0.054217,0.511271,...,1,13,?,19995,498,0,0.104418,0.104478,0.0,3472
21,http://www.pimpthatsnack.com/full.php,Pimp That Snack Snack Search Join the mailing ...,culture_politics,0.208574,2.388489,0.733813,0.496403,0.302158,0.086331,0.723183,...,0,86,0,825,139,1,0.093525,0.125000,1.0,4644
22,http://www.etsy.com/treasury/MTA0NjAxODZ8NzkwM...,,culture_politics,0.165873,1.488235,0.408451,0.140845,0.014085,0.000000,21.000000,...,0,43,0,1818,213,2,0.267606,0.315789,0.0,3528
24,http://warehouse.carlh.com/article_157/,February 4th 2008 Copy the line below to link ...,arts_entertainment,0.361629,2.000000,0.086957,0.000000,0.000000,0.000000,0.485349,...,0,4,0,2814,23,1,0.173913,0.095164,1.0,3744


In [29]:
cleaned_data.iloc[16, 1]

'In a discovery that has stunned even those behind it scientists at a Toronto hospital say they have proof the body s nervous system helps trigger diabetes opening the door to a potential near cure of the disease that affects millions of Canadians Diabetic mice became healthy virtually overnight after researchers injected a substance to counteract the effect of malfunctioning pain neurons in the pancreas I couldn t believe it said Dr Michael Salter a pain expert at the Hospital for Sick Children and one of the scientists Mice with diabetes suddenly didn t have diabetes any more The researchers caution they have yet to confirm their findings in people but say they expect results from human studies within a year or so Any treatment that may emerge to help at least some patients would likely be years away from hitting the market But the excitement of the team from Sick Kids whose work is being published today in the journal Cell is almost palpable I ve never seen anything like it said Dr 

It is clear that this is a news article and its 'isNews' value was set to '?'

This just tells us that we'll have to closely analyze each entry with '?' for isNews to determine whether they're a news page or not

Or we can consider this to be an outlier and assume that its mostly non-news page whenever isNews = '?'

A separate NLP model can be trained to predict whether that page isNews or not given the description of the page. But this can be very unreliable plus we don't have class label 0 entries for isNews in the given dataset. So it makes more sense to straight manually label each entry for whether it is news or not.

Similar manual labelling can be done for isFrontPageNews as well


These same trends were observed on the test data separately as well

### Analyzing isFrontPageNews column for ? values

In [30]:
cleaned_data[cleaned_data['isFrontPageNews'] == '?']['isNews'].value_counts()

?    1247
Name: isNews, dtype: int64

The entries which have ? values in isFrontPageNews also have ? values in isNews

### Analyzing isNews column for ? values

In [31]:
cleaned_data[cleaned_data['isNews'] == '?']['webpageDescription']

0        polyvore is the best place to discover or sta...
1       Speed Air Man--David Belle david belle speed a...
4       Barbecued Chicken Chow Siew from The Exotic Ki...
6       Copyright 2011 InsidersHealth com All Rights R...
7       Well the world is made of peanut butter now po...
                              ...                        
1471                                                     
1474                                                     
1475    Save to your Collections Sorry for the inconve...
1477    outerbanxchic posted 10 14 2011 Delicious and ...
1478    Summer always signifies the beginning of steak...
Name: webpageDescription, Length: 2842, dtype: object

### Analyzing the alchemy_category column

In [32]:
cleaned_data['alchemy_category'].value_counts()

?                     2341
recreation            1229
arts_entertainment     941
business               880
health                 506
sports                 380
culture_politics       343
computer_internet      296
science_technology     289
gaming                  76
religion                72
law_crime               31
unknown                  6
weather                  4
Name: alchemy_category, dtype: int64

In [33]:
cleaned_data[cleaned_data['alchemy_category'] == 'law_crime'].head()

,url,webpageDescription,alchemy_category,alchemy_category_score,avgLinkWordLength,AvglinkWithOneCommonWord,AvglinkWithTwoCommonWord,AvglinkWithThreeCommonWord,AvglinkWithFourCommonWord,redundancyMeasure,...,lengthyDomain,hyperlinkToAllWordsRatio,isFrontPageNews,alphanumCharCount,linksCount,wordCount,parametrizedLinkRatio,spellingErrorsRatio,label,id
66,http://vii2012.com/2012/06/accidental-90s-nick...,Your smiling at me is my daily dose of magic S...,law_crime,0.505332,3.666667,0.545455,0.454545,0.318182,0.227273,0.621094,...,0,51,0,306,22,4,0.000000,0.161290,0.0,6771
377,http://www.independent.co.uk/news/world/americ...,google ad client ca pub 5964551156905038 if re...,law_crime,0.818603,2.068460,0.579572,0.187648,0.054632,0.033254,0.432215,...,1,53,0,3828,421,14,0.076010,0.093960,1.0,3319
639,http://newsfeed.time.com/2012/01/09/reading-wh...,Monday s links talk surprising birthdays body ...,law_crime,0.752151,2.972222,0.553333,0.193333,0.100000,0.073333,0.504981,...,1,57,0,1595,150,8,0.126667,0.087108,0.0,452
847,http://www.insidethekaganoffkitchen.com/2010/0...,by Rachel on April 11 2010 I just love a good ...,law_crime,0.246582,1.904255,0.392523,0.242991,0.121495,0.112150,0.522953,...,1,19,0,3475,107,2,0.009346,0.089928,1.0,3203
1334,http://www.magnoliarouge.com/2012/08/outfit-of...,AboutWelcome to Magnolia Rouge a collection of...,law_crime,0.313761,1.300971,0.258929,0.071429,0.035714,0.017857,0.675141,...,0,45,0,1034,112,4,0.241071,0.136986,0.0,1628


### This cell actually shows you how the alchemy_category can be misclassified

Because some of these pages are clearly not related to law_crime category

This tells us that alchemy_category and the corresponding score cannot be relied on too much and perhaps we can try thresholding off the basis of alchemy_category_score, where we only retain those categories where the score is high, like 60%



In [34]:
cleaned_data[cleaned_data['alchemy_category_score'].apply(lambda x: 0 if x == '?' else float(x)) > 0.6]

,url,webpageDescription,alchemy_category,alchemy_category_score,avgLinkWordLength,AvglinkWithOneCommonWord,AvglinkWithTwoCommonWord,AvglinkWithThreeCommonWord,AvglinkWithFourCommonWord,redundancyMeasure,...,lengthyDomain,hyperlinkToAllWordsRatio,isFrontPageNews,alphanumCharCount,linksCount,wordCount,parametrizedLinkRatio,spellingErrorsRatio,label,id
5,http://www.sleepdisordersguide.com/blog/good-n...,You have to pay high price if you are not gett...,health,0.953935,3.697917,0.848485,0.636364,0.191919,0.101010,0.382585,...,1,23,0,6061,99,16,0.010101,0.105439,1.0,6940
6,http://www.insidershealth.com/natural_cure/poi...,Copyright 2011 InsidersHealth com All Rights R...,health,0.893828,1.691781,0.469799,0.147651,0.026846,0.006711,0.522134,...,0,57,0,961,149,4,0.006711,0.086957,1.0,5416
12,http://sugarcrafter.net/2009/08/20/sesame-chic...,August 20 2009 Print E mail Filed under Asian ...,arts_entertainment,0.722537,2.573529,0.451220,0.195122,0.146341,0.097561,0.487435,...,1,19,0,3447,82,2,0.329268,0.073770,1.0,6194
16,http://www.nationalpost.com/news/story.html?id...,In a discovery that has stunned even those beh...,health,0.603181,3.261649,0.653595,0.369281,0.196078,0.140523,0.442116,...,1,44,0,5791,306,2,0.075163,0.098795,0.0,4794
19,http://bleacherreport.com/articles/1319130-100...,100 Ashley Judd Dario Franchitti Superfan Poli...,arts_entertainment,0.85,2.649573,0.496503,0.237762,0.118881,0.055944,0.463007,...,1,7,0,18053,143,4,0.041958,0.116240,0.0,5947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,http://www.deepsouthdish.com/2010/10/macaroni-...,As soon as I saw this Mac n Cheese Soup recipe...,recreation,0.65858,1.125320,0.295122,0.151220,0.078049,0.043902,0.482424,...,1,27,0,5241,410,4,0.209756,0.094808,NaN,1428
1473,http://www.nanny.net/blog/18-blogs-featuring-t...,Posted on June 5 2013 by admin in NannyMillion...,health,0.625404,4.294737,0.845361,0.701031,0.360825,0.144330,0.466958,...,1,34,0,3456,97,11,0.020619,0.119048,NaN,3262
1474,http://www.lanascooking.com/2011/07/08/roasted...,,business,0.681418,1.640000,0.367521,0.102564,0.034188,0.017094,21.000000,...,1,14,0,4994,117,3,0.119658,0.250000,NaN,3336
1476,http://www.laweekly.com/bestof/2011/award/best...,Best Belly Dance Workout 2011 Swerve Studio s ...,arts_entertainment,0.943693,1.829787,0.512690,0.279188,0.055838,0.020305,0.547304,...,1,56,0,1425,197,5,0.446701,0.056122,NaN,738


There are in total 2733 rows where we have alchemy_category_score > 0.6, which means a 60% confidence in the fact that the alchemy_category is correct. 

In total we have 7394 entries out of which 2341 are ? entries and 2733 entries with high confidence, so we have 7394 - 2341 - 2733 = 2320 entries with tagged alchemy_category but low confidence

In [35]:
cleaned_data[cleaned_data['alchemy_category'] == '?']

,url,webpageDescription,alchemy_category,alchemy_category_score,avgLinkWordLength,AvglinkWithOneCommonWord,AvglinkWithTwoCommonWord,AvglinkWithThreeCommonWord,AvglinkWithFourCommonWord,redundancyMeasure,...,lengthyDomain,hyperlinkToAllWordsRatio,isFrontPageNews,alphanumCharCount,linksCount,wordCount,parametrizedLinkRatio,spellingErrorsRatio,label,id
0,http://www.polyvore.com/cgi/home?id=1389651,polyvore is the best place to discover or sta...,?,?,1.916667,0.047619,0.007937,0.000000,0.000000,0.803797,...,0,34,0,682,126,1,0.531746,0.142857,1.0,3711
1,http://www.youtube.com/watch?v=ippMPPu6gh4,Speed Air Man--David Belle david belle speed a...,?,?,1.257576,0.141026,0.000000,0.000000,0.000000,1.142857,...,0,12,0,3008,78,1,0.628205,0.000000,1.0,7222
11,http://www.chow.com/recipes/13499-creamy-carro...,Difficulty Easy Total Time 55 mins Active Time...,?,?,1.804000,0.436090,0.086466,0.007519,0.003759,0.476190,...,1,25,0,6611,266,4,0.406015,0.068306,1.0,5156
15,http://www.whattoexpect.com/preconception/heal...,Next time you re in the bathroom crack open yo...,?,?,2.476562,0.558824,0.250000,0.095588,0.014706,0.450391,...,1,34,0,3185,136,9,0.044118,0.144550,1.0,4988
23,http://www.washingtonpost.com/lifestyle/wellne...,The oil is the healthiest part of a nut contai...,?,?,2.919403,0.632312,0.281337,0.130919,0.077994,0.440646,...,1,42,0,6962,359,9,0.128134,0.094203,0.0,2242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,http://www.healthguru.com/applications/quiz/qz...,Weird Facts on Drugs Quiz HealthGuru Question ...,?,?,2.243094,0.622951,0.327869,0.120219,0.027322,0.858025,...,1,78,?,730,183,6,0.016393,0.132075,NaN,3106
1469,http://news.menshealth.com/touch-at-your-own-p...,It s a wonder your hand doesn t melt off every...,?,?,0.000000,0.000000,0.000000,0.000000,0.000000,0.512821,...,0,0,0,0,161,12,0.124224,0.080357,NaN,4313
1471,http://dentalcalgary.org/,,?,?,1.363636,0.230769,0.153846,0.000000,0.000000,21.000000,...,0,1,?,4594,13,0,0.000000,0.500000,NaN,6131
1475,http://www.marthastewart.com/255456/billys-cho...,Save to your Collections Sorry for the inconve...,?,?,2.236559,0.676768,0.232323,0.090909,0.020202,0.413333,...,1,36,?,1933,99,4,0.060606,0.105263,NaN,2867


Similar to the isNews labelling, the ? values can fall into any one of the given categories which can be inferred pretty quickly given the context. This can either be manually filled in or a separate NLP classifier can be created.

It also makes sense that once we have finished labelling, the alchemy_category_score column doesn't make sense as by that point we'll only leave those entries in alchemy_category for which we have very high confidence

### Analyzing the numerical columns in the dataset

Basically analyzing the output of the describe() call on the dataset

In [36]:
merged_data.describe()

,avgLinkWordLength,AvglinkWithOneCommonWord,AvglinkWithTwoCommonWord,AvglinkWithThreeCommonWord,AvglinkWithFourCommonWord,redundancyMeasure,embedRatio,framebased,frameTagRatio,domainLink,...,imageTagRatio,lengthyDomain,hyperlinkToAllWordsRatio,alphanumCharCount,linksCount,wordCount,parametrizedLinkRatio,spellingErrorsRatio,label,id
count,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.0,7395.000000,7395.000000,...,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,7395.000000,5916.000000,7395.000000
mean,2.761823,0.468230,0.214080,0.092062,0.049262,2.255103,-0.103750,0.0,0.056423,0.021231,...,0.275709,0.660311,30.077079,5716.598242,178.754564,4.960649,0.172864,0.101221,0.517241,3697.000000
std,8.619793,0.203133,0.146743,0.095978,0.072629,5.704313,0.306545,0.0,0.041446,0.144162,...,1.919320,0.473636,20.393101,8875.432430,179.466198,3.233111,0.183286,0.079231,0.499745,2134.896953
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,0.000000,0.000000,...,-1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.602062,0.340370,0.105263,0.022222,0.000000,0.442616,0.000000,0.0,0.028502,0.000000,...,0.025900,0.000000,14.000000,1579.000000,82.000000,3.000000,0.040984,0.068739,0.000000,1848.500000
50%,2.088235,0.481481,0.202454,0.068627,0.022222,0.483680,0.000000,0.0,0.045775,0.000000,...,0.083051,1.000000,25.000000,3500.000000,139.000000,5.000000,0.113402,0.089312,1.000000,3697.000000
75%,2.627451,0.616604,0.300000,0.133333,0.065065,0.578227,0.000000,0.0,0.073459,0.000000,...,0.236700,1.000000,43.000000,6377.000000,222.000000,7.000000,0.241299,0.112376,1.000000,5545.500000
max,363.000000,1.000000,1.000000,0.980392,0.980392,21.000000,0.250000,0.0,0.444444,1.000000,...,113.333333,1.000000,100.000000,207952.000000,4997.000000,22.000000,1.000000,1.000000,1.000000,7394.000000


avgLinkWordLength has a max value of 363 which is a bit outlier-ish considering both the median and mean are around the 2-3 mark.

However avgLinkWordLength measures the average number of words in a webpage and its sensible that a particular page can have 363 words in it. So you cannot consider it as an invalid value or such.

Simple outlier fixing like setting the outlier values to the Q3 quartile is also valid.

Regardless, it will be beneficial if feature standardization is done on all these numeric columns.

embedRatio column is described to contain the "count of tags" but the column has negative values so it becomes difficult to interpret what "count of tags" refers to. It is important to note that the max value in the column is 0.25 which does seem to indicate that this is a low value column.

framebased seems to have just 0 values, so that can be dropped

domainLink (binary column) seems to be skewed toward the 0 class.

hyperlinkToAllWordsRatio (percentage column) has values in range 0 to 100, so erroneous or outlier values there.

alphanumCharCount understandly has very high values. However min value is 0 but this is also possible because some webpages in the dataset have only images or videos hosted on them.

linksCount, wordCount, parameterizedLinkRatio and spellingErrorsRatio also seem to have appropriate values


In [37]:
 merged_data['framebased'].value_counts()

0    7395
Name: framebased, dtype: int64

The framebased column is completely 0. So we can just drop it

In [38]:
merged_data.drop('framebased', axis=1, inplace=True)

### Analyzing tagRatio column

In [41]:
merged_data['tagRatio'].describe()

count    7395.000000
mean        0.233778
std         0.052487
min         0.045564
25%         0.201061
50%         0.230564
75%         0.260770
max         0.716883
Name: tagRatio, dtype: float64

Everything looks alright here

### Analyzing domainLink column

In [39]:
merged_data['domainLink'].value_counts()

0    7238
1     157
Name: domainLink, dtype: int64

In [40]:
test_data['domainLink'].value_counts()

0    1436
1      43
Name: domainLink, dtype: int64

Huge class imbalance in both train and test data, but oversampling doesn't make much sense. At this point it is more sensible to straight up drop the column entirely but this can be tested later. Train model using this column and without using this column.

### Analyzing lengthyDomain column

In [42]:
merged_data['lengthyDomain'].value_counts()

1    4883
0    2512
Name: lengthyDomain, dtype: int64

Everything looks alright here